In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

In [5]:
df =  pd.read_csv('cleaningStep1.gzip',compression='gzip',parse_dates=True,index_col='date')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10751921 entries, 2012-11-25 to 2012-04-10
Data columns (total 16 columns):
airline               object
airline_code          int64
departure_airport     object
departure_state       object
departure_lat         float64
departure_lon         float64
arrival_airport       object
arrival_state         object
arrival_lat           float64
arrival_lon           float64
departure_schedule    float64
departure_actual      int64
departure_delay       int64
arrival_schedule      float64
arrival_actual        int64
arrival_delay         int64
dtypes: float64(6), int64(5), object(5)
memory usage: 1.4+ GB


In [7]:
df['departure_delay'] = df['departure_delay'].astype(int)
df['arrival_delay'] = df['arrival_delay'].astype(int)

In [8]:
df = data.reset_index()

In [9]:
df.head()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
0,2012-11-25,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1755.0,1805,10,1931.0,1948,17
1,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1755.0,1752,-3,1931.0,1929,-2
2,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1445.0,1442,-3,1623.0,1615,-8
3,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,2005.0,2007,2,2140.0,2139,-1
4,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1630.0,1624,-6,1808.0,1805,-3


In [10]:
def makeDateTime(date,timeInInt):
    hour = math.floor(timeInInt//100)
    extraday = False
    if(hour==24):
        hour = 0
        extraday = True
        
    dateTime = datetime.datetime(date.year,date.month,date.day,hour,math.floor(timeInInt%100),0)
    
    if(extraday):
        dateTime = dateTime + datetime.timedelta(days = 1)
        
    return dateTime

In [11]:
def makeDateTimeWithDelay(dateTime, delayInMinutes):
    actual = dateTime
    delay = datetime.timedelta(minutes= math.floor(delayInMinutes))
    return actual-delay

In [12]:
df['departure_actual_datetime'] = np.nan
df['departure_schedule_datetime'] = np.nan
df['arrival_actual_datetime'] = np.nan
df['arrival_schedule_datetime'] = np.nan

In [13]:
def fillInRows(row):
    base_date = row['date']
    arrival_actual = row['arrival_actual']
    departure_actual = row['departure_actual']
    arrival_delay = row["arrival_delay"]
    departure_delay = row["departure_delay"]
    
    
    row["departure_actual_datetime"] = makeDateTime(base_date,departure_actual)
    row['arrival_actual_datetime'] = makeDateTime(base_date,arrival_actual)
    
    row['departure_schedule_datetime'] = makeDateTimeWithDelay(makeDateTime(base_date,departure_actual),departure_delay)
    row['arrival_schedule_datetime'] = makeDateTimeWithDelay(makeDateTime(base_date,arrival_actual),arrival_delay)
    return row


df = df.apply(fillInRows,axis=1)

In [14]:
df.tail()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,...,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay,departure_actual_datetime,departure_schedule_datetime,arrival_actual_datetime,arrival_schedule_datetime
10751916,2012-03-01,AA,19805,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,2020.0,2143,83,2300.0,16,76,2012-03-01 21:43:00,2012-03-01 20:20:00,2012-03-01 00:16:00,2012-02-29 23:00:00
10751917,2012-04-23,UA,19977,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,705.0,702,-3,953.0,950,-3,2012-04-23 07:02:00,2012-04-23 07:05:00,2012-04-23 09:50:00,2012-04-23 09:53:00
10751918,2012-03-18,UA,19977,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,1615.0,1612,-3,1906.0,1856,-10,2012-03-18 16:12:00,2012-03-18 16:15:00,2012-03-18 18:56:00,2012-03-18 19:06:00
10751919,2012-04-30,AA,19805,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,640.0,635,-5,920.0,917,-3,2012-04-30 06:35:00,2012-04-30 06:40:00,2012-04-30 09:17:00,2012-04-30 09:20:00
10751920,2012-04-10,AA,19805,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,640.0,659,19,920.0,947,27,2012-04-10 06:59:00,2012-04-10 06:40:00,2012-04-10 09:47:00,2012-04-10 09:20:00


In [15]:
df.to_csv('cleaningStep2.gzip',compression='gzip')